In [1]:
search_for = 134
start_from = 0
threads = 10
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.05130791664123535
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 634149671
type: [1, 1, 1, 1, 134] 134
cases of this type: 322417936
100000 14.47243486288599
200000 26.805585241584982
300000 26.453181687681116
400000 28.778985111215427
500000 28.566346106690993
600000 35.256652194305026
700000 37.75246701402547
800000 42.536349294080736
900000 47.235708543100124
1000000 47.56722716814478
1100000 47.36372318035579
1200000 34.458604832000646
1300000 25.39016118868864
1400000 46.03705983455843
1500000 46.78273617923258
1600000 47.1176091989542
1700000 47.103678580483475
1800000 47.46338569730813
1900000 48.98846665578753
2000000 47.31192091540212
2100000 47.074071501888334
2200000 47.40276852498716
2300000 47.53128333453701
2400000 39.043418985699375
2500000 19.905907857090504
2600000 45.53962882888843
2700000 45.239003675628084
2800000 44.88091552551899
2900000 44.18976008002471
3000000 44.95235

29800000 48.01903267013969
29900000 47.70298988393009
30000000 46.039389294486
30100000 23.941193851892542
30200000 36.98876622582361
30300000 45.8900717593455
30400000 47.743959111887236
30500000 48.328301334271764
30600000 47.62321687227521
30700000 47.45130483668162
30800000 48.83150587816468
30900000 48.99074036286783
31000000 49.48096895396108
31100000 23.81650974820574
31200000 45.96681712408901
31300000 50.94733067980523
31400000 48.52801214820943
31500000 41.640897388802436
31600000 25.14842296486356
31700000 47.49663977211964
31800000 48.49130391446513
31900000 47.63683150470159
32000000 47.44865636810157
32100000 48.14165554947075
32200000 46.339730968262415
32300000 45.77672926050561
32400000 43.82934732967656
32500000 25.738075235582954
32600000 34.002454084755314
32700000 43.3868032962324
32800000 45.17727271879714
32900000 46.114607785979246
33000000 47.25417261737336
33100000 45.33455177622009
33200000 48.45659364020978
33300000 48.387923632099216
33400000 47.38443423393

59900000 47.50811333445079
60000000 46.75123542916659
60100000 46.61826240755083
60200000 47.22282360256166
60300000 45.3566953896247
60400000 46.03531124422983
60500000 47.00873709104309
60600000 36.239515131888226
60700000 27.980862999376495
60800000 44.28025217218328
60900000 43.93392195220195
61000000 43.0607714395293
61100000 41.431324791067134
61200000 41.02589019254937
61300000 39.83876329289837
61400000 13.89648709638759
61500000 39.98650697474402
61600000 40.31604154639193
61700000 41.32830925612447
61800000 40.597908880657535
61900000 43.00189758856775
62000000 43.021275747117365
62100000 37.10139451043258
62200000 29.344697180311446
62300000 47.513042169870545
62400000 45.763244216416794
62500000 44.83919841989394
62600000 45.92383342628268
62700000 48.26640255255078
62800000 46.74834954218834
62900000 47.881282552221485
63000000 45.389835146118635
63100000 18.933343704781368
63200000 45.30741173857455
63300000 44.577112161706964
63400000 43.22745193035549
63500000 41.791915

90000000 40.448970957671484
90100000 37.50645773618908
90200000 38.48572065307967
90300000 12.834339060596177
90400000 38.84569868714131
90500000 39.06195677718753
90600000 36.90370258577852
90700000 38.18618166035297
90800000 20.73946869756297
90900000 34.9210865207816
91000000 40.8055403782599
91100000 39.442638145314355
91200000 43.22211942712288
91300000 41.818793947349384
91400000 41.19974363763145
91500000 42.93950793812912
91600000 44.05149562504019
91700000 42.15629449829411
91800000 43.097010148295546
91900000 41.445483408294166
92000000 41.93683311515671
92100000 33.94105358893573
92200000 19.46011325840125
92300000 38.778227566362155
92400000 37.697665644858795
92500000 39.01533352569225
92600000 40.42523623823811
92700000 9.689972224149786
92800000 41.46914438591764
92900000 38.16063052260663
93000000 39.14910408107404
93100000 37.56918266773691
93200000 17.74073540624664
93300000 37.99343208149135
93400000 42.1156004789393
93500000 40.60791340785747
93600000 41.81537828662

119400000 35.07732158744563
119500000 19.085784418389803
119600000 35.92134520907569
119700000 36.952055742345635
119800000 35.013380259668466
119900000 37.92165837590863
120000000 38.99187176015996
120100000 39.34217932299914
120200000 40.798546305099876
120300000 40.11282514344215
120400000 39.905706182410285
120500000 42.19753677592785
120600000 40.58780747879777
120700000 39.303489739197005
120800000 37.935570860065866
120900000 38.24043424951203
121000000 35.94799365638088
121100000 37.34492607499325
121200000 28.325654582029635
121300000 22.748574305657247
121400000 35.40417596760781
121500000 28.344935880887256
121600000 19.5401349458174
121700000 35.00480603123577
121800000 31.81835997831326
121900000 21.088710190475584
122000000 37.52107566063418
122100000 37.539777382626
122200000 35.50114047027968
122300000 37.60179485059404
122400000 37.94669802507325
122500000 40.3264071402883
122600000 40.40266434436497
122700000 40.379727096087045
122800000 39.59955192837795
122900000 39

...

494100000 6.274648771087972
494200000 6.218317974542548
494300000 6.328614567666646
494400000 5.763203046604834
494500000 4.683546312617919
494600000 5.39320376803568
494700000 6.588011812347786
494800000 6.691046351567929
494900000 6.584369485414084
495000000 6.7360593364176085
495100000 3.191918497614344
495200000 6.713053710537567
495300000 6.441593900524195
495400000 6.472795985691494
495500000 6.243866726560092
495600000 5.496810463896336
495700000 5.763390488353782
495800000 5.040383892787669
495900000 6.5312894343196
496000000 6.501771041443713
496100000 6.531599410449567
496200000 6.489383240275937
496300000 3.230510953997286
496400000 6.288106999392403
496500000 6.345944633506346
496600000 6.351544317212318
496700000 6.082569615049193
496800000 4.80236801173818
496900000 5.903090587153588
497000000 4.935877747853413
497100000 6.859674451196431
497200000 6.262764244080658
497300000 6.561445093234324
497400000 6.2731439525351735
497500000 3.2841192003631
497600000 6.039238468971

523400000 5.435491349679226
523500000 4.928874000278073
523600000 4.7622210237823435
523700000 5.266977183607997
523800000 4.926788437319785
523900000 3.826413411124259
524000000 2.519619344840122
524100000 3.10321208579183
524200000 5.343336376943677
524300000 4.786923278363681
524400000 4.610791630453287
524500000 5.0478023776257634
524600000 5.323948753087927
524700000 4.740898975212814
524800000 4.594943895248501
524900000 5.0869310192075154
525000000 4.783665829122858
525100000 3.760876649988726
525200000 2.59847100069648
525300000 3.1356677584109125
525400000 5.12608032348129
525500000 4.443848026043604
525600000 4.56254590669297
525700000 4.66466049732599
525800000 5.542242830145216
525900000 4.784705195689995
526000000 4.652475586848778
526100000 4.947114216390884
526200000 4.920813815439522
526300000 3.788138691060172
526400000 1.8234828360824167
526500000 3.676290207068542
526600000 5.222211668712964
526700000 4.571323647461995
526800000 4.7771519951099615
526900000 4.6662934

552300000 4.156504549328878
552400000 3.6990796445609098
552500000 2.9113140129829005
552600000 3.262049079064681
552700000 3.668441139820561
552800000 3.179728610935366
552900000 2.4426675646791
553000000 3.5778725591718588
553100000 3.70520440819593
553200000 2.8715527968633534
553300000 3.9488315977768877
553400000 4.043908390771006
553500000 4.019786196114558
553600000 3.686799845117795
553700000 2.56356468197036
553800000 3.5667558649647724
553900000 3.5657263786686375
554000000 3.16946249478119
554100000 2.30422370262701
554200000 3.471665337114333
554300000 3.5661185127173005
554400000 2.9113228846969887
554500000 3.8938784474200143
554600000 3.971787708203805
554700000 3.8678117332598125
554800000 3.855032525187125
554900000 2.3041995116196157
555000000 3.7799009140052484
555100000 3.616377333323469
555200000 3.2116429584861437
555300000 2.157401794499476
555400000 3.3987259210511587
555500000 3.595484052111701
555600000 2.6590248918016464
555700000 4.080042157312407
555800000 

579900000 1.9975545731230921
580000000 2.2807384037808873
580100000 2.3795351515328944
580200000 1.9676271408560535
580300000 1.7896553122946885
580400000 1.906765008639685
580500000 1.9048508312525265
580600000 2.5176120021206576
580700000 2.509077443358058
580800000 1.7763177298203314
580900000 1.7964776172690318
581000000 1.874517132953894
581100000 1.7903098727862485
581200000 2.5212451333466306
581300000 2.52585564590436
581400000 1.7151751145586611
581500000 1.8184121835641147
581600000 1.8182164023402512
581700000 1.7152264192062907
581800000 2.4508608247848116
581900000 2.5428339981619836
582000000 1.6712556441358382
582100000 2.0763539945592284
582200000 2.2968636510347844
582300000 1.91846497928124
582400000 2.4211724283582856
582500000 2.460950062312595
582600000 1.6354813540374364
582700000 1.847539988020602
582800000 1.7666407624181686
582900000 1.6682752795168267
583000000 2.364387787816957
583100000 2.4247789615045785
583200000 1.6315015927677758
583300000 1.863116134986

608400000 0.8932422554816041
608500000 1.1415401905912181
608600000 0.7048707678523899
608700000 1.1219196201461572
608800000 1.034998163388765
608900000 1.03149533695482
609000000 0.8902027951746405
609100000 1.1375853058298302
609200000 0.6936597034193913
609300000 1.1000103659654543
609400000 1.0867746100486582
609500000 0.909009108989948
609600000 0.8642755405473503
609700000 1.1136299213381469
609800000 0.6727253225420415
609900000 1.1016312678502036
610000000 1.1360036960678763
610100000 0.7737738554140507
610200000 0.8757332454133921
610300000 1.08268655484436
610400000 0.6824593767220358
610500000 1.0604756046471648
610600000 1.0671999607692213
610700000 0.7160338225067556
610800000 0.9000426949330879
610900000 1.0687629897392008
611000000 0.6856586303419033
611100000 1.0689267481186397
611200000 1.0814091891984867
611300000 0.6401767436694563
611400000 0.9213385285166475
611500000 1.040271431811978
611600000 0.6610618722017348
611700000 1.0427439155746625
611800000 1.041648059